In [1]:
!pip install visual-automata

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.2 MB/s eta 0:00:00
  Created wheel for colormath: filename=colormath-3.0.0-py3-none-any.whl size=39405 sha256=f23ff1ac8dba3d267e83db7cbeb08e3724fc02

In [3]:
import re

try:
    with open("brown_nouns.txt", "r") as f:
        nouns = [line.strip().lower() for line in f if line.strip() and line.strip().isalpha()]
    print(f"Successfully loaded and cleaned {len(nouns)} nouns.")
    print(f"Sample: {nouns[:10]}")
except FileNotFoundError:
    print("Error: brown_nouns.txt not found. Please ensure the file is in the correct directory.")
    nouns = ['fax', 'faxes', 'blitz', 'blitzes', 'try', 'tries', 'boy', 'boys', 'cat', 'cats', 'class', 'classes', 'watch', 'watches']

Successfully loaded and cleaned 201654 nouns.
Sample: ['investigation', 'primary', 'election', 'evidence', 'irregularities', 'place', 'jury', 'presentments', 'charge', 'election']


In [4]:
def analyze_word_fst(word):
    if not word:
        return "Invalid Word"

    if word.endswith('ies'):
        if len(word) > 3:
            root = word[:-3] + 'y'
            return f"{root}+N+PL"
        else:
            return "Invalid Word"

    if word.endswith('es'):
        if len(word) > 2 and word[-3] in 'sxz':
            root = word[:-2]
            return f"{root}+N+PL"
        if len(word) > 3 and word[-4:-2] in ['ch', 'sh']:
            root = word[:-2]
            return f"{root}+N+PL"

    if word.endswith('s'):
        if word.endswith('ys') and len(word) > 2:
             return f"{word} = Invalid Word"
        if word.endswith('ss'):
             return f"{word}+N+SG"
        root = word[:-1]
        if root.endswith(('s', 'x', 'z')) or root.endswith(('ch', 'sh')):
            return f"{word} = Invalid Word"
        return f"{root}+N+PL"

    return f"{word}+N+SG"

In [5]:
test_words = ["fox", "foxes", "watch", "watches", "try", "tries", "bag", "bags", "boy", "boys", "blitz", "blitzes", "class", "foxs"]
print("--- FST Simulation Results ---")
for w in test_words:
    print(f"{w:<10} -> {analyze_word_fst(w)}")

--- FST Simulation Results ---
fox        -> fox+N+SG
foxes      -> fox+N+PL
watch      -> watch+N+SG
watches    -> watch+N+PL
try        -> try+N+SG
tries      -> try+N+PL
bag        -> bag+N+SG
bags       -> bag+N+PL
boy        -> boy+N+SG
boys       -> boys = Invalid Word
blitz      -> blitz+N+SG
blitzes    -> blitz+N+PL
class      -> class+N+SG
foxs       -> foxs = Invalid Word


In [8]:
results = {}
for noun in nouns:
    results[noun] = analyze_word_fst(noun)

print("--- Sample of Full Analysis ---")
for k, v in list(results.items())[:10]:
    print(f"{k:<10} -> {v}")

output_file = "noun_analysis_output.txt"
with open(output_file, "w") as f:
    for word, analysis in results.items():
        f.write(f"{word} = {analysis}\n")

print(f"\nAnalysis complete. Results saved to {output_file}")

--- Sample of Full Analysis ---
investigation -> investigation+N+SG
primary    -> primary+N+SG
election   -> election+N+SG
evidence   -> evidence+N+SG
irregularities -> irregularity+N+PL
place      -> place+N+SG
jury       -> jury+N+SG
presentments -> presentment+N+PL
charge     -> charge+N+SG
praise     -> praise+N+SG

Analysis complete. Results saved to noun_analysis_output.txt


In [15]:
from visual_automata.fa.dfa import VisualDFA

states = {"q0", "q_c", "q_s", "q_y", "q_special", "q_h", "q_i", "q_e", "q_s_plural", "q_trap"}
input_symbols = {chr(ord('a') + i) for i in range(26)}
final_states = {"q0", "q_s", "q_h", "q_y", "q_special", "q_s_plural"}

transitions = {
    'q0': {'c': 'q_c', 's': 'q_s', 'y': 'q_y', 'x': 'q_special', 'z': 'q_special'},
    'q_c': {'h': 'q_h'},
    'q_s': {'h': 'q_h'},
    'q_y': {'i': 'q_i'},
    'q_special': {'e': 'q_e'},
    'q_h': {'e': 'q_e'},
    'q_i': {'e': 'q_e'},
    'q_e': {'s': 'q_s_plural'},
}

for state in states:
    if state not in transitions:
        transitions[state] = {}

    for symbol in input_symbols:
        if symbol not in transitions[state]:
            if state == 'q0':
                transitions[state][symbol] = 'q0'
            elif state in ['q_s_plural', 'q_trap']:
                transitions[state][symbol] = 'q_trap'
            else:
                transitions[state][symbol] = 'q_trap'

dfa = VisualDFA(
    states=states,
    input_symbols=input_symbols,
    transitions=transitions,
    initial_state='q0',
    final_states=final_states
)

print("--- Updated and Corrected DFA ---")
dfa.show_diagram(filename="corrected_dfa.png")
print("Successfully created and saved corrected_dfa.png")

--- Updated and Corrected DFA ---
Successfully created and saved corrected_dfa.png
